In [1]:
from netgen.occ import *

import sys
sys.path.append("../build/rigid_body_FEM_GGL")
from rigid_body_FEM_GGL import *
import rigid_body_FEM_GGL.bla as bla

import pythreejs as p3

from IPython.display import display
import ipywidgets as widgets

### Helper Functions for body setup

In [2]:
def extract_vertices(obj: TopoDS_Shape):
    "extracts a p3js compatible vertex list from a netgen.occ TopoDS_Shape"
    
    data = obj._webgui_data()["Bezier_trig_points"]
    
    # for every face, each of the verts arrays holds one vertex
    verts1 = data[0]
    verts2 = data[1]
    verts3 = data[2]
    
    # corresponding normals
    normals1 = data[3]
    normals2 = data[4]
    normals3 = data[5]

    combined_vertices = []
    for i in range(0, len(verts1), 4):
        combined_vertices.append(verts1[i : i+3])
        combined_vertices.append(verts2[i : i+3])
        combined_vertices.append(verts3[i : i+3])

    combined_normals = []
    for i in range(0, len(normals1), 3):
        combined_normals.append(normals1[i : i+3])
        combined_normals.append(normals2[i : i+3])
        combined_normals.append(normals3[i : i+3])
            
    return combined_vertices, combined_normals


def body_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"

    # important: move the center of mass into the origin
    obj = obj.Move((-obj.center[0], -obj.center[1], -obj.center[2]))
    
    # copy the inertia matrix from netgen
    #inertia_matrix = bla.Matrix(3,3)
    #for i in range(3):
    #    for j in range(3):
    #        inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    #center_of_mass = bla.Vector(3)
    #for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    body = RigidBody_FEM_GGL()
    #for i in range(3) : body.center[i] = obj.center[i]
    #body.mass = obj.mass
    #body.inertia = inertia_matrix
    #body.recalcMassMatrix()

    body.vertices, body.normals = extract_vertices(obj)
    
    return body

## Setup System

In [3]:
# set up OCC CAD model
# center of mass is automatically moved to origin by body_from_solid
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))

In [4]:
# set up physics simulation environment
rbs = RBS_FEM_GGL()
rbs.gravity = (0, 0, 9.81)

# set up physics simulation object for cube
r1 =  body_from_solid(box)

r2 = body_from_solid(box)
# r2.momentumRot = (0.01, 0, 0)
p2 = Transformation()
p2.setTranslation(3,3,3)
r2.transformation=p2
print(r2.transformation)

c1 = rbs.add(r1)
print(r1.transformation)
# c1.pos = (0.5, 0.5, 0.5)
c2 = rbs.add(r2)
c2.pos = (-0.5, -0.5, -0.5)
b = Beam(c1,c2)
rbs.add(b)

f1 = rbs.addFix()
f1.pos = (-1, 0, 3)
print(f1.pos)

b = Beam(f1, c1)
rbs.add(b)

rbs.saveState()
rbs.info_RBS()

 Translation: 	3.000000 ,3.000000, , 3.000000
 Rotation: 	1.000000 ,0.000000, , 0.000000
 		0.000000 ,1.000000, , 0.000000
		0.000000 ,0.000000, , 1.000000

 Translation: 	0.000000 ,0.000000, , 0.000000
 Rotation: 	1.000000 ,0.000000, , 0.000000
 		0.000000 ,1.000000, , 0.000000
		0.000000 ,0.000000, , 1.000000

(-1.0, 0.0, 3.0)
Body Info Index: 0
q: 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1
phat: 0, 0, 0, 0, 0, 0
initialq: 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1
initialphat: 0, 0, 0, 0, 0, 0
mass: 1
mass Matrix: 
1 0 0 0 0 0 
0 1 0 0 0 0 
0 0 1 0 0 0 
0 0 0 1 0 0 
0 0 0 0 1 0 
0 0 0 0 0 1 

mass matrix inverse: 
1 0 0 0 0 0 
0 1 0 0 0 0 
0 0 1 0 0 0 
0 0 0 1 0 0 
0 0 0 0 1 0 
0 0 0 0 0 1 

Connected_Bodies: 0 1 
Center: 0, 0, 0
Springs Index: 
Beams Index: 0 1 
Body 0 end!

Body Info Index: 1
q: 3, 3, 3, 1, 0, 0, 0, 1, 0, 0, 0, 1
phat: 0, 0, 0, 0, 0, 0
initialq: 3, 3, 3, 1, 0, 0, 0, 1, 0, 0, 0, 1
initialphat: 0, 0, 0, 0, 0, 0
mass: 1
mass Matrix: 
1 0 0 0 0 0 
0 1 0 0 0 0 
0 0 1 0 0 0 
0 0 0 1 0 

In [5]:
simulate(rbs, 0.15/50, 1)

In [6]:
rbs.info_RBS()

Body Info Index: 0
q: 0.000019, 0.000005, 0.000006, 1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000, 0.000000, 0.000000, 1.000000
phat: 0.012613, 0.003153, 0.004204, 0.000000, 0.000000, 0.000000
initialq: 0.000000, 0.000000, 0.000000, 1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000, 0.000000, 0.000000, 1.000000
initialphat: 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000
mass: 1.000000
mass Matrix: 
1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 
0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 
0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 

mass matrix inverse: 
1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 0.000000 1.000000 0.000000 0

In [7]:
#simulate(rbs, 0.15/50, 1)

In [8]:
#simulate(rbs, 0.15/50, 1)

## Setup Graphics

In [9]:
def appendConnector(c,connectors):
    p = rbs.connectorPos(c)
    if(c.type == 0):
        color = 'green'
    else :
        color = 'black'
    connectors.append(
        p3.Mesh(p3.SphereBufferGeometry(0.2, 16, 16),
             p3.MeshStandardMaterial(color=color),
             position=(p[0], p[1], p[2])))

def initConnectors(l):
    connectors = []
    for s in l:
        cA = s.connectorA
        cB = s.connectorB
        appendConnector(cA,connectors);
        appendConnector(cB,connectors);
    return connectors
    

def updateConnectors(l, connectors):
    for i in range(len(l)):
        cA = l[i].connectorA
        cB = l[i].connectorB
        pA = rbs.connectorPos(cA);
        pB = rbs.connectorPos(cB);
        connectors[2*i].position = (pA[0], pA[1], pA[2])
        connectors[2*i+1].position = (pB[0], pB[1], pB[2])

def positionsOf(l):
    res = []
    for s in l:
        cA = s.connectorA
        cB = s.connectorB
        pA = rbs.connectorPos(cA);
        pB = rbs.connectorPos(cB);
        res.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    return res


connectorsSprings = initConnectors(rbs.springs())
springpos = positionsOf(rbs.springs())

if springpos:
    springgeo = p3.LineSegmentsGeometry(positions=springpos)
    m2 = p3.LineMaterial(linewidth=3, color='cyan')
    springs = p3.LineSegments2(springgeo, m2)

connectorsBeams = initConnectors(rbs.beams())
beampos = positionsOf(rbs.beams())

if beampos:
    beamgeo = p3.LineSegmentsGeometry(positions=beampos)
    m2 = p3.LineMaterial(linewidth=4, color='blue')
    beams = p3.LineSegments2(beamgeo, m2)


In [10]:
view_width = 1000
view_height = 700
buffergeos = []
p3meshes = []

# set up pythreejs 3d objects
for body in rbs.bodies():
    buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(body.vertices), "normal" : p3.BufferAttribute(body.normals)})
    material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
    p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))
    buffergeos.append(buffergeom)
    p3meshes.append(p3mesh)

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")
axesHelper = p3.AxesHelper(5)

# set up scene
scene = p3.Scene(children=[camera, key_light, ambient_light, grid, axesHelper, *p3meshes, *connectorsSprings, *connectorsBeams] + ([] if not springpos else [springs])+ ([] if not beampos else [beams]))
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [11]:
# play/reset button
play = widgets.Play(
    value=0,
    min=0,
    max=10000,
    step=1,
    interval=1,
    description="Press play",
    disabled=False
)

In [12]:
for m in p3meshes:
    m.matrixAutoUpdate = False # make mesh movable

def refresh():
    "updates all pythreejs object transformations"
    for i in range(0, len(rbs.bodies())):
        p3meshes[i].matrix=rbs.bodies()[i].transformation.asTuple()
    updateConnectors(rbs.springs(),connectorsSprings)
    springpos = positionsOf(rbs.springs())
    if springpos:
        springs.geometry = p3.LineSegmentsGeometry(positions=springpos)
    
    updateConnectors(rbs.beams(),connectorsBeams)
    beampos = positionsOf(rbs.beams())
    if beampos:
        beams.geometry = p3.LineSegmentsGeometry(positions=beampos)

def update():
    "update function, gets called every timestep; quasi main loop"
    simulate(rbs, 0.15/2, 1)
    refresh()

def observer(state):
    "event handler for clickable buttons"
    # if there is a change in time
    if state["name"] == "value":
        # it might be a reset
        if str(state["new"]) == "0":
            rbs.reset()
            refresh()
        # or it might be a progress in time
        else:
            update()
    # repeat is used as an alias to reset
    elif state["name"] == "repeat":
        rbs.reset()
        refresh()

play.observe(observer)

## Renderer

In [13]:
display(widgets.HBox([play, widgets.HTML("<b>click-and-drag to rotate, scroll to zoom, right-click-and-drag to move<b>")]))
display(renderer)
refresh()

Renderer(camera=PerspectiveCamera(aspect=1.4285714285714286, position=(10.0, 6.0, 10.0), projectionMatrix=(1.5…